In [1]:
import gym
import numpy as np

from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')

H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-3
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

    # if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
    #   print('ep {}: game finished, reward: {}'.format(episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:


%time hist1 = train_model(env, model, total_episodes=5500)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -20.000000. running mean: -20.000000
episode 2.000000, reward total was -18.000000. running mean: -19.980000
episode 3.000000, reward total was -18.000000. running mean: -19.960200
episode 4.000000, reward total was -19.000000. running mean: -19.950598
episode 5.000000, reward total was -21.000000. running mean: -19.961092
episode 6.000000, reward total was -21.000000. running mean: -19.971481
episode 7.000000, reward total was -21.000000. running mean: -19.981766
episode 8.000000, reward total was -20.000000. running mean: -19.981949
episode 9.000000, reward total was -18.000000. running mean: -19.962129
episode 10.000000, reward total was -21.000000. running mean: -19.972508
episode 11.000000, reward total was -21.000000. running mean: -19.982783
episode 12.000000, reward total was -19.000000. running mean: -19.972955
episode 13.000000, reward total was -21.000000. running mean: -19.983225
episode 14.000000, reward total was -21.000000. running mean

episode 114.000000, reward total was -21.000000. running mean: -20.349441
episode 115.000000, reward total was -20.000000. running mean: -20.345946
episode 116.000000, reward total was -20.000000. running mean: -20.342487
episode 117.000000, reward total was -21.000000. running mean: -20.349062
episode 118.000000, reward total was -19.000000. running mean: -20.335571
episode 119.000000, reward total was -20.000000. running mean: -20.332215
episode 120.000000, reward total was -21.000000. running mean: -20.338893
episode 121.000000, reward total was -20.000000. running mean: -20.335504
episode 122.000000, reward total was -20.000000. running mean: -20.332149
episode 123.000000, reward total was -21.000000. running mean: -20.338828
episode 124.000000, reward total was -20.000000. running mean: -20.335440
episode 125.000000, reward total was -20.000000. running mean: -20.332085
episode 126.000000, reward total was -20.000000. running mean: -20.328764
episode 127.000000, reward total was -

episode 225.000000, reward total was -19.000000. running mean: -20.111024
episode 226.000000, reward total was -20.000000. running mean: -20.109913
episode 227.000000, reward total was -21.000000. running mean: -20.118814
episode 228.000000, reward total was -20.000000. running mean: -20.117626
episode 229.000000, reward total was -21.000000. running mean: -20.126450
episode 230.000000, reward total was -21.000000. running mean: -20.135185
episode 231.000000, reward total was -21.000000. running mean: -20.143833
episode 232.000000, reward total was -20.000000. running mean: -20.142395
episode 233.000000, reward total was -21.000000. running mean: -20.150971
episode 234.000000, reward total was -20.000000. running mean: -20.149461
episode 235.000000, reward total was -21.000000. running mean: -20.157967
episode 236.000000, reward total was -20.000000. running mean: -20.156387
episode 237.000000, reward total was -21.000000. running mean: -20.164823
episode 238.000000, reward total was -

episode 336.000000, reward total was -20.000000. running mean: -19.966986
episode 337.000000, reward total was -21.000000. running mean: -19.977316
episode 338.000000, reward total was -20.000000. running mean: -19.977543
episode 339.000000, reward total was -21.000000. running mean: -19.987768
episode 340.000000, reward total was -20.000000. running mean: -19.987890
episode 341.000000, reward total was -18.000000. running mean: -19.968011
episode 342.000000, reward total was -19.000000. running mean: -19.958331
episode 343.000000, reward total was -17.000000. running mean: -19.928748
episode 344.000000, reward total was -18.000000. running mean: -19.909460
episode 345.000000, reward total was -20.000000. running mean: -19.910366
episode 346.000000, reward total was -20.000000. running mean: -19.911262
episode 347.000000, reward total was -20.000000. running mean: -19.912149
episode 348.000000, reward total was -20.000000. running mean: -19.913028
episode 349.000000, reward total was -

...

episode 5008.000000, reward total was -6.000000. running mean: -4.638596
episode 5009.000000, reward total was -9.000000. running mean: -4.682210
episode 5010.000000, reward total was -4.000000. running mean: -4.675388
episode 5011.000000, reward total was -2.000000. running mean: -4.648634
episode 5012.000000, reward total was -6.000000. running mean: -4.662147
episode 5013.000000, reward total was -1.000000. running mean: -4.625526
episode 5014.000000, reward total was -8.000000. running mean: -4.659271
episode 5015.000000, reward total was -8.000000. running mean: -4.692678
episode 5016.000000, reward total was -12.000000. running mean: -4.765751
episode 5017.000000, reward total was -7.000000. running mean: -4.788094
episode 5018.000000, reward total was -5.000000. running mean: -4.790213
episode 5019.000000, reward total was -5.000000. running mean: -4.792311
episode 5020.000000, reward total was -2.000000. running mean: -4.764387
episode 5021.000000, reward total was -9.000000. r

episode 5121.000000, reward total was -7.000000. running mean: -4.780121
episode 5122.000000, reward total was -11.000000. running mean: -4.842320
episode 5123.000000, reward total was -11.000000. running mean: -4.903897
episode 5124.000000, reward total was 7.000000. running mean: -4.784858
episode 5125.000000, reward total was -7.000000. running mean: -4.807009
episode 5126.000000, reward total was -5.000000. running mean: -4.808939
episode 5127.000000, reward total was -7.000000. running mean: -4.830850
episode 5128.000000, reward total was -7.000000. running mean: -4.852541
episode 5129.000000, reward total was -4.000000. running mean: -4.844016
episode 5130.000000, reward total was -11.000000. running mean: -4.905576
episode 5131.000000, reward total was -5.000000. running mean: -4.906520
episode 5132.000000, reward total was 2.000000. running mean: -4.837455
episode 5133.000000, reward total was -11.000000. running mean: -4.899080
episode 5134.000000, reward total was -7.000000. 

episode 5234.000000, reward total was -6.000000. running mean: -5.458244
episode 5235.000000, reward total was -10.000000. running mean: -5.503662
episode 5236.000000, reward total was -7.000000. running mean: -5.518625
episode 5237.000000, reward total was -9.000000. running mean: -5.553439
episode 5238.000000, reward total was 7.000000. running mean: -5.427905
episode 5239.000000, reward total was -8.000000. running mean: -5.453626
episode 5240.000000, reward total was -9.000000. running mean: -5.489089
episode 5241.000000, reward total was -3.000000. running mean: -5.464198
episode 5242.000000, reward total was 6.000000. running mean: -5.349556
episode 5243.000000, reward total was 1.000000. running mean: -5.286061
episode 5244.000000, reward total was -10.000000. running mean: -5.333200
episode 5245.000000, reward total was 11.000000. running mean: -5.169868
episode 5246.000000, reward total was -3.000000. running mean: -5.148170
episode 5247.000000, reward total was -14.000000. ru

episode 5347.000000, reward total was -5.000000. running mean: -5.215736
episode 5348.000000, reward total was -1.000000. running mean: -5.173579
episode 5349.000000, reward total was 1.000000. running mean: -5.111843
episode 5350.000000, reward total was 7.000000. running mean: -4.990724
episode 5351.000000, reward total was -7.000000. running mean: -5.010817
episode 5352.000000, reward total was 16.000000. running mean: -4.800709
episode 5353.000000, reward total was -5.000000. running mean: -4.802702
episode 5354.000000, reward total was -1.000000. running mean: -4.764675
episode 5355.000000, reward total was -10.000000. running mean: -4.817028
episode 5356.000000, reward total was -6.000000. running mean: -4.828858
episode 5357.000000, reward total was -10.000000. running mean: -4.880569
episode 5358.000000, reward total was 4.000000. running mean: -4.791764
episode 5359.000000, reward total was -11.000000. running mean: -4.853846
episode 5360.000000, reward total was 3.000000. run

episode 5460.000000, reward total was 8.000000. running mean: -4.804721
episode 5461.000000, reward total was -5.000000. running mean: -4.806674
episode 5462.000000, reward total was -3.000000. running mean: -4.788607
episode 5463.000000, reward total was -7.000000. running mean: -4.810721
episode 5464.000000, reward total was -7.000000. running mean: -4.832614
episode 5465.000000, reward total was -2.000000. running mean: -4.804288
episode 5466.000000, reward total was -3.000000. running mean: -4.786245
episode 5467.000000, reward total was -7.000000. running mean: -4.808382
episode 5468.000000, reward total was -11.000000. running mean: -4.870298
episode 5469.000000, reward total was -12.000000. running mean: -4.941595
episode 5470.000000, reward total was 3.000000. running mean: -4.862179
episode 5471.000000, reward total was 4.000000. running mean: -4.773558
episode 5472.000000, reward total was -12.000000. running mean: -4.845822
episode 5473.000000, reward total was -9.000000. ru